In [1]:
import pandas as pd
import numpy as np
import os
import io

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

from nltk import sent_tokenize
from nltk import word_tokenize

In [25]:
searchList = ['FinTech','TechFin','digital','technology','Blockchain',
              'distributed','Bitcoin','ICO','cryptocurrency','mobile','online',
              'cyber','InsurTech','RegTech','micro']

searchList = [x.lower() for x in searchList]

In [3]:
input_folder = r'U:\\My Documents\\Python\\Text Mining\\PDF Extraction\files'

file_name = os.listdir(input_folder)
file_path = [os.path.join(input_folder, x) for x in file_name]

In [4]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

if __name__ == '__main__':
    doc_list = list(map(lambda x: extract_text_from_pdf(x), file_path))
    print(len(doc_list)) #     print(extract_text_from_pdf(file_path[0]))

27


In [6]:
doc_list = list(map(lambda x: sent_tokenize(x), doc_list))

In [26]:
def search_document(doc, word_l= searchList, n_sentences = 5):
    '''return +/- n_sentences around key words in list word_l'''
    
    result = []
    
    for counter, sent in enumerate(doc):
        sent_lower = word_tokenize(sent.lower())
        if any(w in word_l for w in sent_lower):
            result.append(doc[counter - n_sentences : counter + n_sentences])    
        
    return result

In [27]:
pos_sent = [search_document(doc = x) for x in doc_list]

title = [x[1] for x in doc_list]

In [29]:
output = pd.DataFrame(data={'title': title, 'sentences': pos_sent, 'file': file_name})

output_long = output.apply(lambda x: pd.Series(x['sentences']), axis = 1).stack().reset_index(level=1, drop = True)

output_long.name = 'context'

output_long = pd.DataFrame(output_long)

In [49]:
final_output = pd.merge(output, output_long, how = 'outer', left_index=True, right_index=True)

In [52]:
final_output[['title','file','context']].to_csv('output.csv')

In [ ]:
# # def find_key_words(document, word_l = searchList):
    

# import spacy

# !python -m spacy download en

# nlp = spacy.load('en')